In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
211,AFG,Asia,Afghanistan,2020-08-08,37015.0,78.0,1307.0,9.0,950.850,2.004,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
364,ALB,Europe,Albania,2020-08-08,6151.0,135.0,189.0,1.0,2137.397,46.911,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
581,DZA,Africa,Algeria,2020-08-08,34155.0,529.0,1282.0,9.0,778.887,12.064,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-07,France,197078.0,0.0,30344.0,0.0,0.0,0.0
0,2020-08-07,China,88566.0,0.0,4687.0,0.0,0.0,0.0
0,2020-08-07,Italy,249667.0,0.0,35218.0,0.0,0.0,0.0
0,2020-08-07,Spain,312397.0,0.0,28510.0,0.0,0.0,0.0
0,2020-08-07,United States,4964849.0,0.0,161430.0,0.0,0.0,0.0
0,2020-08-07,World,19439206.0,0.0,721745.0,0.0,0.0,0.0
0,2020-08-07,United Kingdom,309198.0,0.0,46522.0,0.0,0.0,0.0
0,2020-08-07,Germany,215193.0,0.0,9196.0,0.0,0.0,0.0
0,2020-08-07,Iran,323790.0,0.0,18103.0,0.0,0.0,0.0
0,2020-08-07,Turkey,238321.0,0.0,5816.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-08,France,197078.0,197921.0,30344.0,30324.0,-843.0,20.0
0,2020-08-08,China,88566.0,88580.0,4687.0,4681.0,-14.0,6.0
0,2020-08-08,Italy,249667.0,249756.0,35218.0,35190.0,-89.0,28.0
0,2020-08-07,Spain,312397.0,314362.0,28510.0,28503.0,-1965.0,7.0
0,2020-08-08,United States,4964849.0,4941796.0,161430.0,161356.0,23053.0,74.0
0,2020-08-08,World,19439206.0,19357085.0,721745.0,721174.0,82121.0,571.0
0,2020-08-08,United Kingdom,309198.0,309005.0,46522.0,46511.0,193.0,11.0
0,2020-08-08,Germany,215193.0,215336.0,9196.0,9195.0,-143.0,1.0
0,2020-08-08,Iran,323790.0,322567.0,18103.0,18132.0,1223.0,-29.0
0,2020-08-08,Turkey,238321.0,238450.0,5816.0,5813.0,-129.0,3.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")